In [ ]:
from keras.applications import VGG16
conv_base = VGG16(weights='imagenet',include_top=False,input_shape=(150, 150, 3))

In [ ]:
conv_base.summary()

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

baseDir = '/home/chen/LiChiChang/2020SpringThesis/dataset/CatDog/shrink'
trainDir = os.path.join(baseDir, 'train')
validationDir = os.path.join(baseDir, 'validation')
testDir = os.path.join(baseDir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batchSize = 20

def extractFeature(dir, sampleCount):
    features = np.zeros(shape=(sampleCount,4,4,512))
    labels = np.zeros(shape=(sampleCount))
    generator = datagen.flow_from_directory(dir, target_size=(150,150), batch_size=batchSize, class_mode='binary')
    i = 0
    for inputsBatch, labelsBatch in generator:
        featuresBatch = conv_base.predict(inputsBatch)
        features[i * batchSize: (i + 1)*batchSize] = featuresBatch
        labels[i * batchSize: (i + 1)*batchSize] = labelsBatch
        i += 1
        if i*batchSize >= sampleCount:
            break
    return features, labels

trainFeatures, trainLabels = extractFeature(trainDir, 2000)
validationFeatures, validationLabels = extractFeature(validationDir, 1000)
testFeatures, testLabels = extractFeature(testDir, 1000)

trainFeatures = np.reshape(trainFeatures, (2000, 4*4*512))
validationFeatures = np.reshape(validationFeatures, (1000, 4*4*512))
testFeatures = np.reshape(testFeatures, (1000, 4*4*512))

In [ ]:
from keras import models, layers, optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4*4*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5), loss='binary_crossentropy', metrics=['acc'])

history = model.fit(trainFeatures, trainLabels, epochs=30, batch_size=20, validation_data=(validationFeatures,validationLabels))

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
validationAcc = history.history['val_acc']

loss = history.history['loss']
validationLoss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='training acc')
plt.plot(epochs, validationAcc, 'b', label='validation acc')
plt.title('training and validation accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, 'bo', label='training loss')
plt.plot(epochs, validationLoss, 'b', label='validation loss')
plt.title('training and validation loss')
plt.legend()
plt.show()


In [ ]:
'''
Listing 5.20
Combine the 2 models into 1
'''

from keras import models, layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()
print('before freezing the convnet:' , len(model.trainable_weights))
conv_base.trainable = False
print('after freezing the convnet:' , len(model.trainable_weights))


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

trainDatagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
trainGenerator = trainDatagen.flow_from_directory(trainDir, target_size=(150,150), batch_size=20, class_mode='binary')

validationDatagen = ImageDataGenerator(rescale=1./255)#validation and test dataset shouldn't be augmented
validationGenerator = validationDatagen.flow_from_directory(validationDir, target_size=(150,150), batch_size=20, class_mode='binary')

model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])

history = model.fit_generator(trainGenerator, steps_per_epoch=100, epochs=30, validation_data=validationGenerator, validation_steps=50)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
validationAcc = history.history['val_acc']

loss = history.history['loss']
validationLoss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='training acc')
plt.plot(epochs, validationAcc, 'b', label='validation acc')
plt.title('training and validation accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, 'bo', label='training loss')
plt.plot(epochs, validationLoss, 'b', label='validation loss')
plt.title('training and validation loss')
plt.legend()
plt.show()

In [ ]:
conv_base.trainable = True
set_trainable = False

for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable == True
    layer.trainable = set_trainable

model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-5), metrics=['acc'])

history = model.fit_generator(trainGenerator, steps_per_epoch=100, epochs=100, validation_data=validationGenerator, validation_steps=50)

In [1]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

def smooth_curve(points, factor=0.8):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points
plt.plot(epochs,smooth_curve(acc), 'bo', label='Smoothed training acc')
plt.plot(epochs,smooth_curve(val_acc), 'b', label='Smoothed validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs,smooth_curve(loss), 'bo', label='Smoothed training loss')
plt.plot(epochs,smooth_curve(val_loss), 'b', label='Smoothed validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

NameError: name 'history' is not defined